In [1]:
import os
import glob
import random
import tqdm

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image

In [3]:
import tensorflow as tf
tf.__version__

'2.13.0'

In [4]:
%cd /Users/bw/Dataset/MIPI_demosaic_hybridevs/train

/Users/bw/Dataset/MIPI_demosaic_hybridevs/train


In [ ]:
%cd /Users/bw/Dataset/MIPI_demosaic_hybridevs/viz

In [5]:
%pwd

'/Users/bw/Dataset/MIPI_demosaic_hybridevs/train'

In [6]:
!ls -la

total 39268176
drwxr-xr-x     17 bw  staff         544  1 28 18:57 .
drwxr-xr-x      6 bw  staff         192  2  3 01:54 ..
-rw-r--r--@     1 bw  staff        8196  1 28 19:14 .DS_Store
drwxrwxr-x@   802 bw  staff       25664  1 12 18:10 gt
-rw-r--r--@     1 bw  staff  3531585291  1 23 10:55 gt.zip
drwxrwxr-x@     6 bw  staff         192  1 24 13:39 hybirdevs_demosaic_start_here
-rw-r--r--@     1 bw  staff     2851880  1 23 10:49 hybirdevs_demosaic_start_here.zip
drwxrwxr-x@  1602 bw  staff       51264  1 24 12:28 input
-rw-r--r--@     1 bw  staff  1974760586  1 23 10:52 input.zip
-rw-r--r--      1 bw  staff  7097563963  1 26 23:40 pairs.zip
drwxr-xr-x@ 65535 bw  staff     2427968  1 28 14:15 pairs256
-rw-r--r--      1 bw  staff  7498514672  1 28 18:57 pairs256.zip
drwxr-xr-x  65535 bw  staff     9063488  1 28 18:49 pairs_intp
drwxrwxr-x@     6 bw  staff         192  1 13 00:06 scoring_program
-rw-r--r--@     1 bw  staff        2298  1 23 10:49 scoring_program.zip
drwxr-xr-x@    23 bw 

In [7]:
binfiles = glob.glob('input/*.bin')
binfiles.sort()
print(len(binfiles))

800


In [8]:
binfiles[0:4]

['input/0001.bin', 'input/0002.bin', 'input/0003.bin', 'input/0004.bin']

In [9]:
def read_bin(img_path):
    """
    Read bin data
    """
    img_data = np.fromfile(img_path, dtype=np.uint16)
    w = int(img_data[0])
    h = int(img_data[1])
    assert w * h == img_data.size - 2
    # quad = np.clip(img_data[2:].reshape([h, w]).astype(np.float32), 0, 1023)
    quad = np.clip(img_data[2:].reshape([h, w]).astype(np.uint16), 0, 1023)
    return quad

In [ ]:
for binfile in binfiles:
    print(binfile)
    input_quad = read_bin(binfile)
    # plt.imshow(input_quad)
    npyname = binfile.split('.bin')[0] + '.npy'
    print(npyname, input_quad.shape)
    with open(npyname, 'wb') as f:
        np.save(f, input_quad)

In [10]:
gt_files = glob.glob("gt/*.png")
input_files = glob.glob("input/*.npy")
gt_files.sort()
input_files.sort()

In [11]:
print(len(gt_files), len(input_files))

800 800


In [14]:
!mkdir single

In [15]:
%ls -la

total 39268176
drwxr-xr-x     19 bw  staff         608  2  6 00:54 ./
drwxr-xr-x      6 bw  staff         192  2  3 01:54 ../
-rw-r--r--@     1 bw  staff        8196  1 28 19:14 .DS_Store
drwxrwxr-x@   802 bw  staff       25664  1 12 18:10 gt/
-rw-r--r--@     1 bw  staff  3531585291  1 23 10:55 gt.zip
drwxrwxr-x@     6 bw  staff         192  1 24 13:39 hybirdevs_demosaic_start_here/
-rw-r--r--@     1 bw  staff     2851880  1 23 10:49 hybirdevs_demosaic_start_here.zip
drwxrwxr-x@  1602 bw  staff       51264  1 24 12:28 input/
-rw-r--r--@     1 bw  staff  1974760586  1 23 10:52 input.zip
-rw-r--r--      1 bw  staff  7097563963  1 26 23:40 pairs.zip
drwxr-xr-x@ 65535 bw  staff     2427968  1 28 14:15 pairs256/
-rw-r--r--      1 bw  staff  7498514672  1 28 18:57 pairs256.zip
drwxr-xr-x  65535 bw  staff     9063488  1 28 18:49 pairs_intp/
drwxrwxr-x@     6 bw  staff         192  1 13 00:06 scoring_program/
-rw-r--r--@     1 bw  staff        2298  1 23 10:49 scoring_program.zip
drwxr-xr-x@  

# Image Slicing - train

In [17]:
resol = 128
for idx,  (gfile, ifile) in enumerate(zip(gt_files, input_files)):
#     if idx<320:
#          continue

    print(idx, gfile, ifile)

    gt_img = np.array(Image.open(gfile))
    in_img = np.load(ifile)

    print(type(gt_img), gt_img.shape, type(in_img), in_img.shape)


    ny = np.ceil(in_img.shape[0] / resol ).astype(int)
    nx = np.ceil(in_img.shape[1] / resol ).astype(int)

    out_folder = 'single_intp'

    name = gfile.split('/')[-1][:-4]
    num = 0
    print(name, )
    for y in range(ny):
        for x in range(nx):
            sy, ey = y*resol, (y+1)*resol
            sx, ex = x*resol, (x+1)*resol

            if (ey>gt_img.shape[0]):
                ey = gt_img.shape[0]
                sy = ey-resol
            if (ex>gt_img.shape[1]):
                ex = gt_img.shape[1]
                sx = ex-resol

            assert resol == ey-sy
            assert resol == ex-sx

            gt_patch = gt_img[sy:ey, sx:ex]
#             in_patch = in_img[sy:ey, sx:ex]
            # print(gt_patch.shape, in_patch.shape)
            
            
#             ################################################################################################
#             ## interpolation
#             # Red
#             for yy in range(1,in_patch.shape[0],4):
#                 for xx in range(1, in_patch.shape[1], 4):
#                     in_patch[yy][xx] = ((in_patch[yy-1][xx]+in_patch[yy][xx-1])/2)

#             # Blue
#             for yy in range(3,in_patch.shape[0],4):
#                 for xx in range(3, in_patch.shape[1], 4):
#                     in_patch[yy][xx] = ((in_patch[yy-1][xx]+in_patch[yy][xx-1])/2)
#             ################################################################################################
            
            

            seq = "_%04d" % num

            name_seq_gt = name + seq + '_gt.npy'
#             name_seq_in = name + seq + '_in.npy'


            name_seq_gt = os.path.join(out_folder,name_seq_gt)
#             name_seq_in = os.path.join(out_folder,name_seq_in)
            # print(name_seq_gt, name_seq_in)

            with open(name_seq_gt, 'wb') as f:
                np.save(f, gt_patch)
#             with open(name_seq_in, 'wb') as f:
#                 np.save(f, in_patch)

            num += 1

    # break

0 gt/0001.png input/0001.npy
<class 'numpy.ndarray'> (1404, 2040, 3) <class 'numpy.ndarray'> (1404, 2040)
0001
1 gt/0002.png input/0002.npy
<class 'numpy.ndarray'> (1848, 2040, 3) <class 'numpy.ndarray'> (1848, 2040)
0002
2 gt/0003.png input/0003.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0003
3 gt/0004.png input/0004.npy
<class 'numpy.ndarray'> (1344, 2040, 3) <class 'numpy.ndarray'> (1344, 2040)
0004
4 gt/0005.png input/0005.npy
<class 'numpy.ndarray'> (2040, 1608, 3) <class 'numpy.ndarray'> (2040, 1608)
0005
5 gt/0006.png input/0006.npy
<class 'numpy.ndarray'> (2040, 1356, 3) <class 'numpy.ndarray'> (2040, 1356)
0006
6 gt/0007.png input/0007.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0007
7 gt/0008.png input/0008.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0008
8 gt/0009.png input/0009.npy
<class 'numpy.ndarray'> (1524, 2040, 3) <class 'numpy.ndarray'> (1524, 2040)
0009
9

74 gt/0075.png input/0075.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0075
75 gt/0076.png input/0076.npy
<class 'numpy.ndarray'> (1572, 2040, 3) <class 'numpy.ndarray'> (1572, 2040)
0076
76 gt/0077.png input/0077.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0077
77 gt/0078.png input/0078.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0078
78 gt/0079.png input/0079.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0079
79 gt/0080.png input/0080.npy
<class 'numpy.ndarray'> (1332, 2040, 3) <class 'numpy.ndarray'> (1332, 2040)
0080
80 gt/0081.png input/0081.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0081
81 gt/0082.png input/0082.npy
<class 'numpy.ndarray'> (1212, 2040, 3) <class 'numpy.ndarray'> (1212, 2040)
0082
82 gt/0083.png input/0083.npy
<class 'numpy.ndarray'> (1164, 2040, 3) <class 'numpy.ndarray'> (1164, 204

147 gt/0148.png input/0148.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0148
148 gt/0149.png input/0149.npy
<class 'numpy.ndarray'> (1752, 2040, 3) <class 'numpy.ndarray'> (1752, 2040)
0149
149 gt/0150.png input/0150.npy
<class 'numpy.ndarray'> (2040, 1164, 3) <class 'numpy.ndarray'> (2040, 1164)
0150
150 gt/0151.png input/0151.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0151
151 gt/0152.png input/0152.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0152
152 gt/0153.png input/0153.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0153
153 gt/0154.png input/0154.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0154
154 gt/0155.png input/0155.npy
<class 'numpy.ndarray'> (2040, 2040, 3) <class 'numpy.ndarray'> (2040, 2040)
0155
155 gt/0156.png input/0156.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (

220 gt/0221.png input/0221.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0221
221 gt/0222.png input/0222.npy
<class 'numpy.ndarray'> (1164, 2040, 3) <class 'numpy.ndarray'> (1164, 2040)
0222
222 gt/0223.png input/0223.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0223
223 gt/0224.png input/0224.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0224
224 gt/0225.png input/0225.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0225
225 gt/0226.png input/0226.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0226
226 gt/0227.png input/0227.npy
<class 'numpy.ndarray'> (2040, 1356, 3) <class 'numpy.ndarray'> (2040, 1356)
0227
227 gt/0228.png input/0228.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0228
228 gt/0229.png input/0229.npy
<class 'numpy.ndarray'> (1296, 2040, 3) <class 'numpy.ndarray'> (

293 gt/0294.png input/0294.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0294
294 gt/0295.png input/0295.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0295
295 gt/0296.png input/0296.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0296
296 gt/0297.png input/0297.npy
<class 'numpy.ndarray'> (1344, 2040, 3) <class 'numpy.ndarray'> (1344, 2040)
0297
297 gt/0298.png input/0298.npy
<class 'numpy.ndarray'> (1224, 2040, 3) <class 'numpy.ndarray'> (1224, 2040)
0298
298 gt/0299.png input/0299.npy
<class 'numpy.ndarray'> (1392, 2040, 3) <class 'numpy.ndarray'> (1392, 2040)
0299
299 gt/0300.png input/0300.npy
<class 'numpy.ndarray'> (1320, 2040, 3) <class 'numpy.ndarray'> (1320, 2040)
0300
300 gt/0301.png input/0301.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0301
301 gt/0302.png input/0302.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (

...

439 gt/0440.png input/0440.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0440
440 gt/0441.png input/0441.npy
<class 'numpy.ndarray'> (1188, 2040, 3) <class 'numpy.ndarray'> (1188, 2040)
0441
441 gt/0442.png input/0442.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0442
442 gt/0443.png input/0443.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0443
443 gt/0444.png input/0444.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0444
444 gt/0445.png input/0445.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0445
445 gt/0446.png input/0446.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0446
446 gt/0447.png input/0447.npy
<class 'numpy.ndarray'> (1692, 2040, 3) <class 'numpy.ndarray'> (1692, 2040)
0447
447 gt/0448.png input/0448.npy
<class 'numpy.ndarray'> (1536, 2040, 3) <class 'numpy.ndarray'> (

512 gt/0513.png input/0513.npy
<class 'numpy.ndarray'> (2040, 2040, 3) <class 'numpy.ndarray'> (2040, 2040)
0513
513 gt/0514.png input/0514.npy
<class 'numpy.ndarray'> (1368, 2040, 3) <class 'numpy.ndarray'> (1368, 2040)
0514
514 gt/0515.png input/0515.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0515
515 gt/0516.png input/0516.npy
<class 'numpy.ndarray'> (1536, 2040, 3) <class 'numpy.ndarray'> (1536, 2040)
0516
516 gt/0517.png input/0517.npy
<class 'numpy.ndarray'> (1536, 2040, 3) <class 'numpy.ndarray'> (1536, 2040)
0517
517 gt/0518.png input/0518.npy
<class 'numpy.ndarray'> (960, 2040, 3) <class 'numpy.ndarray'> (960, 2040)
0518
518 gt/0519.png input/0519.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0519
519 gt/0520.png input/0520.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0520
520 gt/0521.png input/0521.npy
<class 'numpy.ndarray'> (2040, 1368, 3) <class 'numpy.ndarray'> (20

585 gt/0586.png input/0586.npy
<class 'numpy.ndarray'> (2040, 1428, 3) <class 'numpy.ndarray'> (2040, 1428)
0586
586 gt/0587.png input/0587.npy
<class 'numpy.ndarray'> (1368, 2040, 3) <class 'numpy.ndarray'> (1368, 2040)
0587
587 gt/0588.png input/0588.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0588
588 gt/0589.png input/0589.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0589
589 gt/0590.png input/0590.npy
<class 'numpy.ndarray'> (2040, 1356, 3) <class 'numpy.ndarray'> (2040, 1356)
0590
590 gt/0591.png input/0591.npy
<class 'numpy.ndarray'> (1212, 2040, 3) <class 'numpy.ndarray'> (1212, 2040)
0591
591 gt/0592.png input/0592.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0592
592 gt/0593.png input/0593.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0593
593 gt/0594.png input/0594.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (

658 gt/0659.png input/0659.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0659
659 gt/0660.png input/0660.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0660
660 gt/0661.png input/0661.npy
<class 'numpy.ndarray'> (1296, 2040, 3) <class 'numpy.ndarray'> (1296, 2040)
0661
661 gt/0662.png input/0662.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0662
662 gt/0663.png input/0663.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0663
663 gt/0664.png input/0664.npy
<class 'numpy.ndarray'> (1632, 2040, 3) <class 'numpy.ndarray'> (1632, 2040)
0664
664 gt/0665.png input/0665.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0665
665 gt/0666.png input/0666.npy
<class 'numpy.ndarray'> (2040, 1476, 3) <class 'numpy.ndarray'> (2040, 1476)
0666
666 gt/0667.png input/0667.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (

731 gt/0732.png input/0732.npy
<class 'numpy.ndarray'> (1536, 2040, 3) <class 'numpy.ndarray'> (1536, 2040)
0732
732 gt/0733.png input/0733.npy
<class 'numpy.ndarray'> (1308, 2040, 3) <class 'numpy.ndarray'> (1308, 2040)
0733
733 gt/0734.png input/0734.npy
<class 'numpy.ndarray'> (2040, 1356, 3) <class 'numpy.ndarray'> (2040, 1356)
0734
734 gt/0735.png input/0735.npy
<class 'numpy.ndarray'> (1536, 2040, 3) <class 'numpy.ndarray'> (1536, 2040)
0735
735 gt/0736.png input/0736.npy
<class 'numpy.ndarray'> (1452, 2040, 3) <class 'numpy.ndarray'> (1452, 2040)
0736
736 gt/0737.png input/0737.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0737
737 gt/0738.png input/0738.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0738
738 gt/0739.png input/0739.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (1356, 2040)
0739
739 gt/0740.png input/0740.npy
<class 'numpy.ndarray'> (1356, 2040, 3) <class 'numpy.ndarray'> (

In [20]:
!ls -la single

total 14728080
drwxr-xr-x@ 65535 bw  staff  4531808  2  6 01:01 .
drwxr-xr-x     18 bw  staff      576  2  6 01:02 ..
-rw-r--r--@     1 bw  staff     6148  2  6 00:55 .DS_Store
-rw-r--r--@     1 bw  staff    49280  2  6 00:56 0001_0000_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:56 0001_0001_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:56 0001_0002_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:56 0001_0003_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:56 0001_0004_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:56 0001_0005_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:56 0001_0006_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:56 0001_0007_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:56 0001_0008_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:56 0001_0009_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:56 0001_0010_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:56 0001_0011_gt.npy
-rw-r--r--@     1 bw  staff    

-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0126_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0127_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0128_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0129_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0130_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0131_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0132_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0133_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0134_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0135_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0136_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0137_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0138_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0139_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0027_0140_gt.npy
-rw-r--r--

-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0119_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0120_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0121_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0122_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0123_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0124_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0125_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0126_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0127_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0128_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0129_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0130_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0131_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0132_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0058_0133_gt.npy
-rw-r--r--

-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0087_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0088_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0089_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0090_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0091_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0092_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0093_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0094_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0095_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0096_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0097_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0098_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0099_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0100_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0089_0101_gt.npy
-rw-r--r--

-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0157_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0158_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0159_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0160_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0161_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0162_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0163_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0164_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0165_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0166_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0167_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0168_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0169_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0170_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0119_0171_gt.npy
-rw-r--r--

-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0150_0150_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0150_0151_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0150_0152_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0150_0153_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0150_0154_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0150_0155_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0150_0156_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0150_0157_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0150_0158_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0150_0159_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0151_0000_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0151_0001_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0151_0002_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0151_0003_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0151_0004_gt.npy
-rw-r--r--

-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0018_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0019_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0020_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0021_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0022_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0023_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0024_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0025_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0026_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0027_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0028_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0029_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0030_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0031_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:57 0182_0032_gt.npy
-rw-r--r--

-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0002_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0003_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0004_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0005_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0006_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0007_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0008_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0009_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0010_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0011_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0012_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0013_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0014_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0015_gt.npy
-rw-r--r--@     1 bw  staff    49280  2  6 00:58 0213_0016_gt.npy
-rw-r--r--

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [21]:
%pwd

'/Users/bw/Dataset/MIPI_demosaic_hybridevs/train'

# shuffle patch, order is in "order" array

In [26]:
pnames_gt = glob.glob(os.path.join("single", "*_gt.npy"))

pnames_gt.sort()

flen = len(pnames_gt)
print(flen)


order = np.arange(flen)
np.random.shuffle(order)
print(order)



pnames_gt = [ pnames_gt[x] for x in order]


141616
[  9425  27345  23749 ... 101804  24915  85332]


In [27]:
pnames_gt

['single/0053_0113_gt.npy',
 'single/0155_0145_gt.npy',
 'single/0135_0005_gt.npy',
 'single/0124_0126_gt.npy',
 'single/0771_0031_gt.npy',
 'single/0705_0124_gt.npy',
 'single/0164_0012_gt.npy',
 'single/0781_0116_gt.npy',
 'single/0142_0128_gt.npy',
 'single/0635_0066_gt.npy',
 'single/0318_0174_gt.npy',
 'single/0366_0034_gt.npy',
 'single/0510_0068_gt.npy',
 'single/0132_0169_gt.npy',
 'single/0672_0122_gt.npy',
 'single/0107_0115_gt.npy',
 'single/0107_0174_gt.npy',
 'single/0107_0001_gt.npy',
 'single/0403_0100_gt.npy',
 'single/0108_0058_gt.npy',
 'single/0356_0068_gt.npy',
 'single/0101_0176_gt.npy',
 'single/0531_0189_gt.npy',
 'single/0470_0065_gt.npy',
 'single/0048_0050_gt.npy',
 'single/0719_0064_gt.npy',
 'single/0503_0068_gt.npy',
 'single/0654_0060_gt.npy',
 'single/0641_0001_gt.npy',
 'single/0641_0058_gt.npy',
 'single/0183_0110_gt.npy',
 'single/0416_0145_gt.npy',
 'single/0456_0133_gt.npy',
 'single/0134_0215_gt.npy',
 'single/0677_0048_gt.npy',
 'single/0652_0070_g

In [28]:
def write_tfrecords(file_list_gt, idx_split, dataset_name, output_path):

    ext = file_list_gt[0].split('.')[-1]

    tfrecords_name = '%s_%03d.tfrecords' % (dataset_name, idx_split)
    tfrecords_name = os.path.join(output_path, tfrecords_name)

    if os.path.isfile(tfrecords_name):
        print('OHNO -->. tfrecord (%s) exists' % tfrecords_name)
        #return

    print('tfrecord not exists')
    print('--->', tfrecords_name)

    writer = tf.io.TFRecordWriter(tfrecords_name)

    for index, file_names in enumerate(tqdm.tqdm(file_list_gt)):
#         print('%d / %d' % (index+1, len(file_list)))

        file_name_gt = file_names
        # for png

        image_arr_gt = np.load(file_name_gt)

        image_byte_gt = image_arr_gt.tobytes()

        example = tf.train.Example(
                features=tf.train.Features(feature={
                    'image': tf.train.Feature(
                            bytes_list=tf.train.BytesList(value=[image_byte_gt])
                    )
                })
        )



        writer.write(example.SerializeToString())
    writer.close()

In [29]:
def make_tfrecords(flists, num_splits, dataset_name, output_path='.'):

    flist_gt = flists

    assert len(flist_gt) > 0, f'no file is found'
  

    total_len = len(flist_gt)
    split_len = total_len // num_splits

    print(total_len, split_len)
    start_idx = np.array(list(range(num_splits)) )* split_len
    finish_idx= (np.array((list(range(num_splits)) )) + 1) * split_len

    print(start_idx)
    print(finish_idx)
    finish_idx[-1] = total_len

    # exit()
    # print(start_idx)
    # print(finish_idx)

    for idx_split in range(num_splits):
        print(f'... {idx_split+1} / {num_splits}')
        idx_s = start_idx[idx_split]
        idx_f = finish_idx[idx_split]
        write_tfrecords(flist_gt[idx_s:idx_f], idx_split, dataset_name, output_path)

    print('done make_tfrecords')

In [34]:
# make tfrecords
num_of_splits = 20
make_tfrecords(flists=pnames_gt, num_splits=num_of_splits, dataset_name='div2k_tetra_gt', output_path='tfrecords_gt')

141616 7080
[     0   7080  14160  21240  28320  35400  42480  49560  56640  63720
  70800  77880  84960  92040  99120 106200 113280 120360 127440 134520]
[  7080  14160  21240  28320  35400  42480  49560  56640  63720  70800
  77880  84960  92040  99120 106200 113280 120360 127440 134520 141600]
... 1 / 20
tfrecord not exists
---> tfrecords_gt/div2k_tetra_gt_000.tfrecords


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 7080/7080 [00:03<00:00, 1783.71it/s]


... 2 / 20
tfrecord not exists
---> tfrecords_gt/div2k_tetra_gt_001.tfrecords


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 7080/7080 [00:03<00:00, 1827.18it/s]


... 3 / 20
tfrecord not exists
---> tfrecords_gt/div2k_tetra_gt_002.tfrecords


...

... 19 / 20
tfrecord not exists
---> tfrecords_gt/div2k_tetra_gt_018.tfrecords


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 7080/7080 [00:03<00:00, 1889.54it/s]


... 20 / 20
tfrecord not exists
---> tfrecords_gt/div2k_tetra_gt_019.tfrecords


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 7096/7096 [00:04<00:00, 1772.24it/s]

done make_tfrecords


In [33]:
!pwd

/Users/bw/Dataset/MIPI_demosaic_hybridevs/train


In [ ]:
!mkdir tfrecords

In [ ]:
!ls -la

# Image Slicing - valid/test

In [ ]:
%cd /Users/bw/Dataset/MIPI_demosaic_hybridevs/val

In [ ]:
!ls -la

In [ ]:
binfiles = glob.glob('input/*.bin')
binfiles.sort()
print(len(binfiles))